In [1]:
##################################################################################
##### Define all parameters for model tuning
##################################################################################

n_fold = 5
expName = "NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_TPC"
outPath = "Results"
foldName = "folds.pickle"

shuffle = True
seed = None

input_data_folder = "Data_from_Asim"
file = "TPC[100, 0, 0, 0]-st-simplesequence.csv"

monitor = "val_loss"

In [2]:
import os 
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import roc_auc_score, matthews_corrcoef

import math

In [3]:
# print(tf.test.is_gpu_available(cuda_only=True))
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
##################################################################################
##### Build k-fold functions
##################################################################################

## Build the K-fold from dataset
def build_kfold(features, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features, labels):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X_train": X_train,
            "X_test": X_test,
            "y_train":y_train,
            "y_test":y_test
        })
    return kfoldList

In [5]:
##################################################################################
##### define evaluator functions
##################################################################################

def pred2label(y_pred):
    y_pred = np.round(y_pred).astype(int)
    return y_pred

# Prepare Training and Independent data

In [6]:
input_data_file = os.path.join(input_data_folder, file)

data = pd.read_csv(input_data_file, sep=',', header=0)

train_data = data[data['set'] == 'train'].drop('set', axis=1)
independent_data = data[data['set'] == 'test'].drop('set', axis=1)

train_labels = np.array(train_data['labels'])
train_labels = train_labels.reshape((train_labels.shape[0], 1))

train_features = np.array(train_data.drop('labels', axis=1))

indpe_labels = np.array(independent_data['labels'])
indpe_labels = indpe_labels.reshape((indpe_labels.shape[0], 1))

indpe_features = np.array(independent_data.drop('labels', axis=1))

##################################################################################
##### extract data from the current fasta file
##################################################################################

print("\n======================================================================")
print("\nFile:", file)
print("Training Positive:", np.sum(train_labels))
print("Training Negative:", train_labels.shape[0] - np.sum(train_labels))
print("Independent Positive:", np.sum(indpe_labels))
print("Independent Negative:", indpe_labels.shape[0] - np.sum(indpe_labels))
print("Feature size:", train_features[0].shape)

##################################################################################
##### Generate Folds from dataset, and store to file
##################################################################################

## Generate the k-fold dataset
folds = build_kfold(train_features, train_labels, k=n_fold, shuffle=shuffle, seed=seed)

## Write the k-fold dataset to file
foldPath = os.path.join(outPath, expName, "{}fold".format(n_fold))
if(not os.path.isdir(foldPath)):
    os.makedirs(foldPath)
pickle.dump(folds, open(os.path.join(foldPath, foldName), "wb"))



File: TPC[100, 0, 0, 0]-st-simplesequence.csv
Training Positive: 1191
Training Negative: 1191
Independent Positive: 203
Independent Negative: 1022
Feature size: (8000,)


# Model

In [7]:
epochs = 100
batch_size = 16

##################################################################################
##### Function to customize the DLNN architecture with parameters
##################################################################################

def DLNN_Classifier(input_vec_shape,
                    dense_decode_units = 32, ## Dense layer parameters
                    prob = 0.5, learn_rate = 0.0005, loss = 'binary_crossentropy', metrics = 'accuracy'):
    
    beta = 0.001
    
    input1 = tf.keras.layers.Input(shape=input_vec_shape)
    
    ######################################################################################################
    ########  Classifier  ################################################################################
    ######################################################################################################
    
    y = tf.keras.layers.Dense(dense_decode_units, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'relu'
                             )(input1)
    
    y = tf.keras.layers.Dropout(prob)(y)
    
    y = tf.keras.layers.Dense(int(dense_decode_units/2), 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'relu'
                             )(y)
    
    y = tf.keras.layers.Dropout(prob)(y)
    
    y = tf.keras.layers.Dense(1, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'sigmoid')(y)

    ## Generate Model from input and output
    model = tf.keras.models.Model(inputs=input1, outputs=y)
    
    ## Compile model
    if(metrics != None):
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
                      loss = loss, metrics = metrics)
    else:
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
                      loss = loss)

    return model

In [8]:
DLNN_Classifier((400,)).summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 400)]             0         
                                                                 
 dense (Dense)               (None, 32)                12832     
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 13,377
Trainable params: 13,377
Non-trainable p

# Training

In [9]:
##################################################################################
##### For each input file, train model and generate different outputs in a structured folder
##################################################################################

## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

## Create and set directory to save model
modelPath = os.path.join(outPath, expName, "{}fold".format(n_fold), "models")
if(not os.path.isdir(modelPath)):
    os.makedirs(modelPath)

##################################################################################
##### TRAIN and PREDICT for every Fold, using models
##################################################################################

# fold counter
i = 0

for fold in folds:

    # adding random shuffling of the dataset for training purpose
    randomized_index_arr = np.arange(fold["X_train"].shape[0])
    randomized_index_arr = np.random.permutation(randomized_index_arr)

    print("\nTrain/Test model on Fold #"+str(i)+".")

    input_size = fold["X_train"][0].shape
    ## Generate model using function
    model = DLNN_Classifier(input_vec_shape = input_size)

    model_file_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    ## Define the model callbacks for early stopping and saving the model. Then train model
    modelCallbacks = [
        tf.keras.callbacks.ModelCheckpoint(model_file_path,
                                           monitor = monitor, verbose = 1, save_best_only = True, 
                                           save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
    ]
    model.fit(x = fold["X_train"][randomized_index_arr], y = fold["y_train"][randomized_index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
              callbacks = modelCallbacks, validation_data = (fold["X_test"], fold["y_test"]))

    model = tf.keras.models.load_model(model_file_path)

    ##################################################################################
    ##### Prediction and metrics for TRAIN dataset
    ##################################################################################

    y_pred = model.predict(fold["X_train"])
    label_pred = pred2label(y_pred)
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_train"], label_pred)
    prec = precision_score(fold["y_train"],label_pred)

    mcc = matthews_corrcoef(fold["y_train"], label_pred)

    conf = confusion_matrix(fold["y_train"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)

    fpr, tpr, thresholds = roc_curve(fold["y_train"], y_pred)
    auc = roc_auc_score(fold["y_train"], y_pred)

    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Train")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)

    ##################################################################################
    ##### Prediction and metrics for TEST dataset
    ##################################################################################

    y_pred = model.predict(fold["X_test"])
    label_pred = pred2label(y_pred)
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_test"], label_pred)
    prec = precision_score(fold["y_test"],label_pred)
    mcc = matthews_corrcoef(fold["y_test"], label_pred)

    conf = confusion_matrix(fold["y_test"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)

    fpr, tpr, thresholds = roc_curve(fold["y_test"], y_pred)
    auc = roc_auc_score(fold["y_test"], y_pred)

    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Test")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)

    i = i+1
    del model
    tf.keras.backend.clear_session()


Train/Test model on Fold #0.
Epoch 1/100
109/120 [==========================>...] - ETA: 0s - loss: 0.7312 - accuracy: 0.5092
Epoch 1: val_loss improved from inf to 0.71480, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_TPC\5fold\models\bestModel-fold0.hdf5
120/120 [==============================] - 1s 5ms/step - loss: 0.7299 - accuracy: 0.5192 - val_loss: 0.7148 - val_accuracy: 0.5912
Epoch 2/100
114/120 [===========================>..] - ETA: 0s - loss: 0.7097 - accuracy: 0.5746
Epoch 2: val_loss improved from 0.71480 to 0.70720, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_TPC\5fold\models\bestModel-fold0.hdf5
120/120 [==============================] - 0s 4ms/step - loss: 0.7095 - accuracy: 0.5732 - val_loss: 0.7072 - val_accuracy: 0.5010
Epoch 3/100
112/120 [===========================>..] - ETA: 0s - loss: 0.7036 - accuracy: 0.5921
Epoch 3: val_loss improved from 0.70720 to 0.70383, saving model to Results\NT_Site_PredNTS_C

120/120 [==============================] - 0s 4ms/step - loss: 0.3667 - accuracy: 0.9906 - val_loss: 0.6813 - val_accuracy: 0.7987
Epoch 26/100
106/120 [=========================>....] - ETA: 0s - loss: 0.3646 - accuracy: 0.9923
Epoch 26: val_loss did not improve from 0.64985
120/120 [==============================] - 0s 4ms/step - loss: 0.3629 - accuracy: 0.9932 - val_loss: 0.6869 - val_accuracy: 0.7945
Epoch 27/100
111/120 [==========================>...] - ETA: 0s - loss: 0.3573 - accuracy: 0.9910
Epoch 27: val_loss did not improve from 0.64985
120/120 [==============================] - 0s 4ms/step - loss: 0.3589 - accuracy: 0.9895 - val_loss: 0.6871 - val_accuracy: 0.7945
Epoch 28/100
112/120 [===========================>..] - ETA: 0s - loss: 0.3570 - accuracy: 0.9905
Epoch 28: val_loss did not improve from 0.64985
120/120 [==============================] - 0s 4ms/step - loss: 0.3569 - accuracy: 0.9906 - val_loss: 0.6947 - val_accuracy: 0.7862
Epoch 29/100
108/120 [================

Epoch 85/100
108/120 [==========================>...] - ETA: 0s - loss: 0.3455 - accuracy: 0.9942
Epoch 85: val_loss did not improve from 0.64985
120/120 [==============================] - 0s 4ms/step - loss: 0.3456 - accuracy: 0.9942 - val_loss: 0.7067 - val_accuracy: 0.7987
Epoch 86/100
119/120 [============================>.] - ETA: 0s - loss: 0.3410 - accuracy: 0.9926
Epoch 86: val_loss did not improve from 0.64985
120/120 [==============================] - 0s 4ms/step - loss: 0.3409 - accuracy: 0.9927 - val_loss: 0.7142 - val_accuracy: 0.7925
Epoch 87/100
120/120 [==============================] - ETA: 0s - loss: 0.3413 - accuracy: 0.9932
Epoch 87: val_loss did not improve from 0.64985
120/120 [==============================] - 0s 4ms/step - loss: 0.3413 - accuracy: 0.9932 - val_loss: 0.7041 - val_accuracy: 0.7945
Epoch 88/100
120/120 [==============================] - ETA: 0s - loss: 0.3421 - accuracy: 0.9921
Epoch 88: val_loss did not improve from 0.64985
120/120 [==============

120/120 [==============================] - 1s 4ms/step - loss: 0.6914 - accuracy: 0.6835 - val_loss: 0.6984 - val_accuracy: 0.7631
Epoch 12/100
111/120 [==========================>...] - ETA: 0s - loss: 0.6823 - accuracy: 0.7843
Epoch 12: val_loss improved from 0.69836 to 0.69785, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_TPC\5fold\models\bestModel-fold1.hdf5
120/120 [==============================] - 0s 4ms/step - loss: 0.6822 - accuracy: 0.7816 - val_loss: 0.6979 - val_accuracy: 0.6730
Epoch 13/100
107/120 [=========================>....] - ETA: 0s - loss: 0.6674 - accuracy: 0.8318
Epoch 13: val_loss improved from 0.69785 to 0.69070, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_TPC\5fold\models\bestModel-fold1.hdf5
120/120 [==============================] - 1s 4ms/step - loss: 0.6664 - accuracy: 0.8341 - val_loss: 0.6907 - val_accuracy: 0.7799
Epoch 14/100
111/120 [==========================>...] - ETA: 0s - loss: 0.6355 - 

Epoch 68/100
109/120 [==========================>...] - ETA: 0s - loss: 0.3368 - accuracy: 0.9943
Epoch 68: val_loss did not improve from 0.66033
120/120 [==============================] - 0s 4ms/step - loss: 0.3374 - accuracy: 0.9948 - val_loss: 0.7102 - val_accuracy: 0.7883
Epoch 69/100
108/120 [==========================>...] - ETA: 0s - loss: 0.3342 - accuracy: 0.9954
Epoch 69: val_loss did not improve from 0.66033
120/120 [==============================] - 0s 4ms/step - loss: 0.3366 - accuracy: 0.9937 - val_loss: 0.7043 - val_accuracy: 0.7987
Epoch 70/100
107/120 [=========================>....] - ETA: 0s - loss: 0.3383 - accuracy: 0.9959
Epoch 70: val_loss did not improve from 0.66033
120/120 [==============================] - 0s 4ms/step - loss: 0.3402 - accuracy: 0.9948 - val_loss: 0.7186 - val_accuracy: 0.7925
Epoch 71/100
105/120 [=========================>....] - ETA: 0s - loss: 0.3366 - accuracy: 0.9964
Epoch 71: val_loss did not improve from 0.66033
120/120 [==============

...

Epoch 67/100
108/120 [==========================>...] - ETA: 0s - loss: 0.3426 - accuracy: 0.9925
Epoch 67: val_loss did not improve from 0.67308
120/120 [==============================] - 0s 4ms/step - loss: 0.3461 - accuracy: 0.9921 - val_loss: 0.7058 - val_accuracy: 0.7878
Epoch 68/100
107/120 [=========================>....] - ETA: 0s - loss: 0.3415 - accuracy: 0.9895
Epoch 68: val_loss did not improve from 0.67308
120/120 [==============================] - 0s 4ms/step - loss: 0.3432 - accuracy: 0.9890 - val_loss: 0.7106 - val_accuracy: 0.7794
Epoch 69/100
110/120 [==========================>...] - ETA: 0s - loss: 0.3379 - accuracy: 0.9909
Epoch 69: val_loss did not improve from 0.67308
120/120 [==============================] - 0s 4ms/step - loss: 0.3400 - accuracy: 0.9906 - val_loss: 0.7097 - val_accuracy: 0.7773
Epoch 70/100
108/120 [==========================>...] - ETA: 0s - loss: 0.3417 - accuracy: 0.9919
Epoch 70: val_loss did not improve from 0.67308
120/120 [==============

Epoch 97/100
112/120 [===========================>..] - ETA: 0s - loss: 0.3420 - accuracy: 0.9927
Epoch 97: val_loss did not improve from 0.67308
120/120 [==============================] - 0s 4ms/step - loss: 0.3424 - accuracy: 0.9927 - val_loss: 0.7124 - val_accuracy: 0.7815
Epoch 98/100
111/120 [==========================>...] - ETA: 0s - loss: 0.3411 - accuracy: 0.9932
Epoch 98: val_loss did not improve from 0.67308
120/120 [==============================] - 0s 4ms/step - loss: 0.3420 - accuracy: 0.9921 - val_loss: 0.7121 - val_accuracy: 0.7815
Epoch 99/100
106/120 [=========================>....] - ETA: 0s - loss: 0.3403 - accuracy: 0.9935
Epoch 99: val_loss did not improve from 0.67308
120/120 [==============================] - 0s 4ms/step - loss: 0.3396 - accuracy: 0.9932 - val_loss: 0.7132 - val_accuracy: 0.7815
Epoch 100/100
108/120 [==========================>...] - ETA: 0s - loss: 0.3396 - accuracy: 0.9919
Epoch 100: val_loss did not improve from 0.67308
120/120 [============

Epoch 22/100
106/120 [=========================>....] - ETA: 0s - loss: 0.3851 - accuracy: 0.9906
Epoch 22: val_loss did not improve from 0.65769
120/120 [==============================] - 0s 4ms/step - loss: 0.3872 - accuracy: 0.9911 - val_loss: 0.6909 - val_accuracy: 0.7899
Epoch 23/100
105/120 [=========================>....] - ETA: 0s - loss: 0.3810 - accuracy: 0.9899
Epoch 23: val_loss did not improve from 0.65769
120/120 [==============================] - 0s 4ms/step - loss: 0.3827 - accuracy: 0.9900 - val_loss: 0.6800 - val_accuracy: 0.8025
Epoch 24/100
109/120 [==========================>...] - ETA: 0s - loss: 0.3743 - accuracy: 0.9920
Epoch 24: val_loss did not improve from 0.65769
120/120 [==============================] - 0s 4ms/step - loss: 0.3768 - accuracy: 0.9916 - val_loss: 0.6764 - val_accuracy: 0.8025
Epoch 25/100
110/120 [==========================>...] - ETA: 0s - loss: 0.3744 - accuracy: 0.9932
Epoch 25: val_loss did not improve from 0.65769
120/120 [==============

Epoch 52/100
111/120 [==========================>...] - ETA: 0s - loss: 0.3356 - accuracy: 0.9949
Epoch 52: val_loss did not improve from 0.65769
120/120 [==============================] - 0s 4ms/step - loss: 0.3354 - accuracy: 0.9953 - val_loss: 0.6992 - val_accuracy: 0.7983
Epoch 53/100
120/120 [==============================] - ETA: 0s - loss: 0.3398 - accuracy: 0.9948
Epoch 53: val_loss did not improve from 0.65769
120/120 [==============================] - 0s 4ms/step - loss: 0.3398 - accuracy: 0.9948 - val_loss: 0.7152 - val_accuracy: 0.7794
Epoch 54/100
120/120 [==============================] - ETA: 0s - loss: 0.3417 - accuracy: 0.9948
Epoch 54: val_loss did not improve from 0.65769
120/120 [==============================] - 0s 4ms/step - loss: 0.3417 - accuracy: 0.9948 - val_loss: 0.6894 - val_accuracy: 0.8025
Epoch 55/100
106/120 [=========================>....] - ETA: 0s - loss: 0.3460 - accuracy: 0.9923
Epoch 55: val_loss did not improve from 0.65769
120/120 [==============

Epoch 82/100
106/120 [=========================>....] - ETA: 0s - loss: 0.3376 - accuracy: 0.9935
Epoch 82: val_loss did not improve from 0.65769
120/120 [==============================] - 0s 4ms/step - loss: 0.3390 - accuracy: 0.9937 - val_loss: 0.7017 - val_accuracy: 0.7983
Epoch 83/100
109/120 [==========================>...] - ETA: 0s - loss: 0.3360 - accuracy: 0.9971
Epoch 83: val_loss did not improve from 0.65769
120/120 [==============================] - 0s 4ms/step - loss: 0.3369 - accuracy: 0.9969 - val_loss: 0.6939 - val_accuracy: 0.7983
Epoch 84/100
107/120 [=========================>....] - ETA: 0s - loss: 0.3387 - accuracy: 0.9936
Epoch 84: val_loss did not improve from 0.65769
120/120 [==============================] - 0s 4ms/step - loss: 0.3384 - accuracy: 0.9942 - val_loss: 0.7017 - val_accuracy: 0.7941
Epoch 85/100
107/120 [=========================>....] - ETA: 0s - loss: 0.3369 - accuracy: 0.9947
Epoch 85: val_loss did not improve from 0.65769
120/120 [==============

## k-fold Training evaluation

In [10]:
evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Test,0.789247,0.780945,0.872406,0.806076,0.772476,0.579975
Train,0.979534,0.976091,0.997441,0.983206,0.975859,0.959131


In [11]:
evaluations_df

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
0,0,Train,0.982677,0.987275,"[0.0, 0.0010504201680672268, 0.061974789915966...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.9822993, 0.98229927, 0.9229434, 0.92243487,...",0.997601,0.977941,0.987408,0.965397
1,0,Test,0.796646,0.814159,"[0.0, 0.0041841004184100415, 0.075313807531380...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9609051, 0.9609051, 0.8788847, 0.87847334, ...",0.881702,0.769874,0.823529,0.594219
2,1,Train,0.977953,0.970072,"[0.0, 0.001049317943336831, 0.1374606505771248...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.9774461, 0.977446, 0.9227576, 0.92231804, 0...",0.998484,0.986359,0.969538,0.956040
3,1,Test,0.794549,0.771318,"[0.0, 0.004201680672268907, 0.0672268907563025...","[0.0, 0.0, 0.0, 0.0041841004184100415, 0.00418...","[1.9782629, 0.97826296, 0.915286, 0.90902233, ...",0.870812,0.836134,0.753138,0.591251
4,2,Train,0.983736,0.979210,"[0.0, 0.001049317943336831, 0.0944386149003148...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.9852929, 0.98529285, 0.9273256, 0.92696553,...",0.998117,0.988458,0.979014,0.967514
5,2,Test,0.781513,0.770161,"[0.0, 0.004201680672268907, 0.1680672268907563...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9871342, 0.9871343, 0.85135996, 0.84992325,...",0.872855,0.802521,0.760504,0.563523
6,3,Train,0.980588,0.976091,"[0.0, 0.001049317943336831, 0.1993704092339979...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.9897163, 0.98971623, 0.9118084, 0.91145086,...",0.997785,0.985310,0.975866,0.961218
7,3,Test,0.787815,0.758491,"[0.0, 0.004201680672268907, 0.0462184873949579...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9568915, 0.95689154, 0.89887637, 0.8988658,...",0.865855,0.844538,0.731092,0.579371
8,4,Train,0.972718,0.967809,"[0.0, 0.001049317943336831, 0.1280167890870933...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.001049317943336831...","[1.9732636, 0.9732636, 0.845738, 0.8451987, 0....",0.995215,0.977964,0.967471,0.945488
9,4,Test,0.785714,0.790598,"[0.0, 0.004201680672268907, 0.0294117647058823...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9399755, 0.93997556, 0.8645771, 0.8628225, ...",0.870807,0.777311,0.794118,0.571509


# Independent data

In [12]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Independent Data performance
##################################################################################

print("\nIndependent evaluation for model.")

# adding random shuffling of the dataset for training purpose
randomized_index_arr = np.arange(train_features.shape[0])
randomized_index_arr = np.random.permutation(randomized_index_arr)

input_size = train_features[0].shape
## Generate model using function
model = DLNN_Classifier(input_vec_shape = input_size)

model_file_path = os.path.join(modelPath, "bestModel-full.hdf5")
## Define the model callbacks for early stopping and saving the model. Then train model
modelCallbacks = [
    tf.keras.callbacks.ModelCheckpoint(model_file_path,
                                       monitor = monitor, verbose = 1, save_best_only = True, 
                                       save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
]
model.fit(x = train_features[randomized_index_arr], y = train_labels[randomized_index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
          callbacks = modelCallbacks, validation_data = (indpe_features, indpe_labels))

model = tf.keras.models.load_model(model_file_path)

##################################################################################
##### Prediction and metrics for Train dataset
##################################################################################

y_pred = model.predict(train_features)
label_pred = pred2label(y_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(train_labels, label_pred)
prec = precision_score(train_labels,label_pred)
mcc = matthews_corrcoef(train_labels, label_pred)

conf = confusion_matrix(train_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(train_labels, y_pred)
auc = roc_auc_score(train_labels, y_pred)

evaluations["Train_Test"].append("Train")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

y_pred = model.predict(indpe_features)
label_pred = pred2label(y_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
auc = roc_auc_score(indpe_labels, y_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

del model
tf.keras.backend.clear_session()


Independent evaluation for model.
Epoch 1/100
149/149 [==============================] - ETA: 0s - loss: 0.7215 - accuracy: 0.5038
Epoch 1: val_loss improved from inf to 0.71394, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_TPC\5fold\models\bestModel-full.hdf5
149/149 [==============================] - 1s 7ms/step - loss: 0.7215 - accuracy: 0.5038 - val_loss: 0.7139 - val_accuracy: 0.1657
Epoch 2/100
141/149 [===========================>..] - ETA: 0s - loss: 0.7041 - accuracy: 0.5678
Epoch 2: val_loss improved from 0.71394 to 0.70830, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_TPC\5fold\models\bestModel-full.hdf5
149/149 [==============================] - 1s 5ms/step - loss: 0.7040 - accuracy: 0.5668 - val_loss: 0.7083 - val_accuracy: 0.1657
Epoch 3/100
145/149 [============================>.] - ETA: 0s - loss: 0.6997 - accuracy: 0.5638
Epoch 3: val_loss improved from 0.70830 to 0.70023, saving model to Results\NT_Site_PredNT

149/149 [==============================] - 1s 4ms/step - loss: 0.3787 - accuracy: 0.9908 - val_loss: 1.1739 - val_accuracy: 0.5763
Epoch 29/100
148/149 [============================>.] - ETA: 0s - loss: 0.3783 - accuracy: 0.9890
Epoch 29: val_loss did not improve from 0.69611
149/149 [==============================] - 1s 5ms/step - loss: 0.3789 - accuracy: 0.9891 - val_loss: 1.2339 - val_accuracy: 0.5576
Epoch 30/100
143/149 [===========================>..] - ETA: 0s - loss: 0.3754 - accuracy: 0.9882
Epoch 30: val_loss did not improve from 0.69611
149/149 [==============================] - 1s 4ms/step - loss: 0.3763 - accuracy: 0.9874 - val_loss: 1.1646 - val_accuracy: 0.5869
Epoch 31/100
135/149 [==========================>...] - ETA: 0s - loss: 0.3755 - accuracy: 0.9884
Epoch 31: val_loss did not improve from 0.69611
149/149 [==============================] - 1s 4ms/step - loss: 0.3759 - accuracy: 0.9887 - val_loss: 1.1476 - val_accuracy: 0.5886
Epoch 32/100
139/149 [================

Epoch 88/100
148/149 [============================>.] - ETA: 0s - loss: 0.3579 - accuracy: 0.9916
Epoch 88: val_loss did not improve from 0.69611
149/149 [==============================] - 1s 5ms/step - loss: 0.3582 - accuracy: 0.9912 - val_loss: 1.2737 - val_accuracy: 0.5739
Epoch 89/100
149/149 [==============================] - ETA: 0s - loss: 0.3595 - accuracy: 0.9929
Epoch 89: val_loss did not improve from 0.69611
149/149 [==============================] - 1s 5ms/step - loss: 0.3595 - accuracy: 0.9929 - val_loss: 1.2078 - val_accuracy: 0.5927
Epoch 90/100
141/149 [===========================>..] - ETA: 0s - loss: 0.3545 - accuracy: 0.9916
Epoch 90: val_loss did not improve from 0.69611
149/149 [==============================] - 1s 4ms/step - loss: 0.3556 - accuracy: 0.9912 - val_loss: 1.3645 - val_accuracy: 0.5535
Epoch 91/100
148/149 [============================>.] - ETA: 0s - loss: 0.3577 - accuracy: 0.9924
Epoch 91: val_loss did not improve from 0.69611
149/149 [==============

In [13]:
evaluations_df = pd.DataFrame.from_dict(evaluations)
evaluations_df

,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
0,Train,0.793031,0.979396,"[0.0, 0.0008396305625524769, 0.115869017632241...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.5768489, 0.57684886, 0.5261054, 0.5259261, ...",0.969916,0.598657,0.987406,0.636095
1,Independent,0.770612,0.312500,"[0.0, 0.0, 0.009852216748768473, 0.00985221674...","[0.0, 0.0009784735812133072, 0.000978473581213...","[1.5563588, 0.55635875, 0.55079776, 0.5391654,...",0.620367,0.320197,0.860078,0.178533
